In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# Download the Sonar dataset
# Method 1: Using UCI ML Repository package
try:
    from ucimlrepo import fetch_ucirepo
    
    # Fetch dataset
    sonar_dataset = fetch_ucirepo(id=151)
    
    # Extract features and target
    X = sonar_dataset.data.features.values
    y = sonar_dataset.data.targets.values
    
except ImportError:
    # Method 2: Direct download
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
    data = pd.read_csv(url, header=None)
    
    # The last column contains the class labels
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values

# Label encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode the target
y_onehot = to_categorical(y_encoded)

# Normalize the input features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Define the K-fold Cross Validator
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# K-fold Cross Validation
fold_no = 1
for train, test in kfold.split(X_normalized, y_encoded):
    # Define the model
    model = Sequential()
    # 60 input features to 1 output node with sigmoid activation
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_normalized[train], y_encoded[train], 
                        epochs=100, 
                        batch_size=5, 
                        verbose=0)
    
    # Evaluate on the test data
    scores = model.evaluate(X_normalized[test], y_encoded[test], verbose=0)
    
    # Print results for this fold
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    
    # Add scores to our lists
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increase fold number
    fold_no += 1

# Print average scores
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')
